In [1]:
# Airbnb stays by country
# SQL Database Querying
# Suppose you work for Airbnb as an analyst. A team has come to you asking for a ranked
# list containing total # stays as well as total revenue, split by country. The time frame 
# they're looking for is all stays that began in 2018. You can sort the list in descending 
# order by revenue.

# The table schemas are provided below. Using SQL, write a query to answer this question.


# Table: property_location_info

# Column Name	Data Type	Description
# property_id	integer	ID of the property location
# country	string	country code of the property location
# city_name	string	name of city (note there can be multiple cities with the same name)
# subregion_name	string	provence, state, or subregion name
# address	string	address of property location

# Table: stays_info

# Column Name	Data Type	Description
# guest_id	integer	ID of guest
# property_id	integer	ID of the property location
# host_id	integer	ID of the host managing the property
# revenue	integer	cost of stay for guest in USD
# date_start	string	start day of stay, format is "YYYY-mm-dd"
# date_end	string	end day of stay, format is "YYYY-mm-dd"
# stay_length	integer	number of days for the stay
# airbnb_revenue	integer	revenue that Airbnb collected on stay


In [2]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime

In [3]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Exception as e:
        print(e)
 
    return conn



In [4]:
def create_tables():
    '''Function to create tables
    Args:
        None
    Returns:
        None
    '''

    conn = create_connection('problem21.db')
    cur = conn.cursor()
    
    cur.execute('DROP TABLE IF EXISTS property_location_info')
    cur.execute('DROP TABLE IF EXISTS stays_info')
    conn.commit()

    create_stays_info = '''
        CREATE TABLE IF NOT EXISTS stays_info(
        guest_id int,
        property_id int,
        host_id int,
        revenue int,
        date_start varchar,
        date_end varchar,
        stay_length int,
        airbnb_revenue int
        );
    '''

    create_property_location_info = '''
        CREATE TABLE IF NOT EXISTS property_location_info(
        property_id int,
        country varchar,
        city_name varchar,
        subregion_name varchar,
        address varchar
        );
    '''
    try:
        cur.execute(create_stays_info)
        cur.execute(create_property_location_info)
        conn.commit()
        print('Tables_Created')
    except Exception as e:
        print('Error Creating Tables')
        print(e)
        conn.rollback()
     
    df = pd.read_sql('SELECT * FROM stays_info', conn)
    print(df)
    cur.close()
    conn.close()

In [5]:
def insert_data():
    '''Function to populate tables with sample data to query against
    Args:
        None
    Returns:
        None
    '''
    #15 items
    stays_info_df = pd.DataFrame({
                        'guest_id':   [1,1,1,2,3,2,3,2,4,3,4,3,1,2,3],
                        'property_id':[1,1,1,1,1,1,1,1,2,2,2,2,2,2,2],
                        'host_id':    [1,2,1,2,1,2,1,2,1,2,1,2,1,2,1],
                        'revenue':    [13,14,13,16,15,17,18,15,12,15,13,17,14,19,15],
                        'date_start':['2017-03-12',
                                    '2018-04-21',
                                    '2017-01-12',
                                    '2018-02-23',
                                    '2017-03-14',
                                    '2018-04-25',
                                    '2017-05-16',
                                    '2018-06-27',
                                    '2017-07-18',
                                    '2018-08-29',
                                    '2017-09-10',
                                    '2018-10-21',
                                    '2017-11-12',
                                    '2018-11-23',
                                    '2018-12-14'
                                   ],
                        'date_end':  ['2017-04-12',
                                    '2018-05-21',
                                    '2017-02-12',
                                    '2018-03-23',
                                    '2017-04-14',
                                    '2018-05-25',
                                    '2017-06-16',
                                    '2018-07-27',
                                    '2017-08-18',
                                    '2018-09-29',
                                    '2017-10-10',
                                    '2018-10-27',
                                    '2017-11-23',
                                    '2018-11-28',
                                    '2018-12-24'],
                        'airbnb_revenue': [3,4,3,6,5,7,8,5,2,5,3,7,4,9,5]
                        })
    
    # get stay_length
    stays_info_df['stay_length'] = (pd.to_datetime(stays_info_df['date_end'], format='%Y-%m-%d') - 
          pd.to_datetime(stays_info_df['date_start'], format='%Y-%m-%d'))/np.timedelta64(1, 'D')
    
    stays_info_df['stay_length'] = stays_info_df['stay_length'].astype('int64')

    #print(stays_info_df.dtypes)
    #print(stays_info_df.head())
    
    conn = create_connection('problem21.db')
    cur = conn.cursor()
    # insert data
    stays_insert_ = '''
    INSERT INTO stays_info(
        guest_id,property_id,host_id,revenue,date_start,date_end,stay_length,airbnb_revenue) 
        VALUES(?,?,?,?,?,?,?,?)
    '''
    
    for _, row in stays_info_df.iterrows():
        stays_values = (row['guest_id'],row['property_id'],row['host_id'],
                  row['revenue'],row['date_start'],row['date_end'],
                  row['stay_length'],row['airbnb_revenue'])
        
        cur.execute(stays_insert_, stays_values)
        
    
    property_location_info_df = pd.DataFrame({
        'property_id':[1,2],
        'country':['US', 'Mexico'],
        'city_name':['LA', 'Cancun'],
        'subregion_name':['CA','Quintana Roo'],
        'address':['######1', '######2'],
            })
    
    property_insert = '''
        INSERT INTO property_location_info(property_id,country, city_name, subregion_name, address)
        VALUES(?,?,?,?,?)
    '''
    
    for _, row in property_location_info_df.iterrows():
        property_values = (row['property_id'], row['country'], row['city_name'], row['subregion_name'], row['address'])
        cur.execute(property_insert, property_values)
        
    conn.commit()
    cur.close()
    conn.close()
    print('Data Inserted')
#insert_data()

In [6]:
def make_populate_data():
    '''Function to create tables and insert sample data'
    Args:
        None
    Returns:
        None
        '''
    create_tables()
    insert_data()

make_populate_data()

Tables_Created
Empty DataFrame
Columns: [guest_id, property_id, host_id, revenue, date_start, date_end, stay_length, airbnb_revenue]
Index: []
Data Inserted


In [7]:
# total # stays as well as total revenue, split by country. The time frame 
# they're looking for is all stays that began in 2018. You can sort the list in descending 
# order by revenue.
def get_stays():
    '''Function returns df with total # stays as well as total revenue, split by country. The time frame 
      is all stays that began in 2018 sorted descending order by revenue.
    Args:
        None
    Returns:
        None
    '''
    
    conn = create_connection('problem21.db')
    cur = conn.cursor()
    
    query = '''
        SELECT
            COUNT(si.revenue) AS total_stays,
            SUM(si.revenue) AS total_revinue,
            pli.country
        FROM
            stays_info  AS si INNER JOIN property_location_info AS pli
            ON si.property_id = PLI.property_id
        WHERE
            strftime('%Y',si.date_start) >=2018
        GROUP BY
            pli.country
    '''
    
    df = pd.read_sql(query, conn)

    print(df)

In [8]:
get_stays()


   total_stays  total_revinue country
0            7            105  Mexico
1            8            121      US
